In [1]:
%matplotlib inline
import paramiko
import getpass
import cv2
import os
import numpy as np
import nibabel as nb
from tqdm import tqdm
import matplotlib.pyplot as plt

/home/simon/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
s = paramiko.SSHClient()
s.set_missing_host_key_policy(paramiko.AutoAddPolicy())
p = getpass.getpass('Password')
s.connect("miplabsrv3.epfl.ch",22,username="snarduzz",password=p,timeout=4)

Password········


In [3]:
def normalize(v):
    m = v.max()
    if(m==0):
        return np.zeros(v.shape)
    else:
        return v/m

def get_ftp_image(path,ssh_client=s):
    local = "/tmp/file.nii"
    sftp = ssh_client.open_sftp()
    with sftp.open(path) as f:
        data = f.read()
        with open(local,"wb") as local_f:
            local_f.write(data)
        img = nb.load(local).get_data()
    return img

In [5]:
directory = "/home/klug/data/preprocessed_original/"

sftp = s.open_sftp()
patients = sftp.listdir(directory)

patient_lesion_size = {}
zeros = {}
tissue = {}
for p in tqdm(patients):
    lesion = get_ftp_image(os.path.join(directory,p,"Neuro_Cerebrale_64Ch","wcoreg_VOI_lesion_{}.nii".format(p)))
    MTT = get_ftp_image(os.path.join(directory,p,"Ct2_Cerebrale","wcoreg_RAPID_MTT_{}.nii".format(p)))
    Tmax = get_ftp_image(os.path.join(directory,p,"Ct2_Cerebrale","wcoreg_RAPID_Tmax_{}.nii".format(p)))
    CBF = get_ftp_image(os.path.join(directory,p,"Ct2_Cerebrale","wcoreg_RAPID_rCBF_{}.nii".format(p)))
    CBV = get_ftp_image(os.path.join(directory,p,"Ct2_Cerebrale","wcoreg_RAPID_rCBV_{}.nii".format(p)))
    
    patient_lesion_size[p] = np.sum(normalize(lesion))
    tissue[p] = np.count_nonzero(MTT)-patient_lesion_size[p]
    zeros[p] = MTT.size-tissue[p]+patient_lesion_size[p]


100%|██████████| 51/51 [06:49<00:00,  8.04s/it]


In [ ]:
print("Lesion shape : {}".format(lesion.shape))
print("MTT shape : {}".format(MTT.shape))
print("CBV shape : {}".format(CBV.shape))
print("CBF shape : {}".format(CBF.shape))
print("Tmax shape : {}".format(Tmax.shape))

In [ ]:
plt.Figure(figsize=[4,7])
plt.hist(patient_lesion_size.values())
plt.title("Distribution of lesion size")
plt.xlabel("Size (voxels)")
plt.ylabel("Total patients")
plt.show()

In [ ]:
plt.Figure(figsize=[4,7])
plt.hist(zeros.values())
plt.title("Distribution of Non-tissue voxels (background)")
plt.xlabel("Size (voxels)")
plt.ylabel("Total patients")
plt.show()

In [ ]:
plt.Figure(figsize=[4,7])
plt.hist(tissue.values())
plt.title("Distribution of healthy tissue voxels (non-lesioned)")
plt.xlabel("Size (voxels)")
plt.ylabel("Total patients")
plt.show()